In [1]:
from bs4 import BeautifulSoup as bs
from lxml import etree
from pprint import pprint
from pyquery import PyQuery as pq
from selenium import webdriver
from urllib.parse import urlencode
import json
import os
import pandas as pd
import re
import requests
import time

In [2]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

In [3]:
region_map = {
    '台北市':'1',
    '基隆市':'2',
    '新北市':'3',
    '新竹市':'4',
    '新竹縣':'5',
    '桃園市':'6',
    '苗栗縣':'7',
    '台中市':'8',
    '彰化縣':'10',
    '南投縣':'11',
    '嘉義市':'12',
    '嘉義縣':'13',
    '雲林縣':'14',
    '台南市':'15',
    '高雄市':'17',
    '屏東縣':'19',
    '宜蘭縣':'21',
    '台東縣':'22',
    '花蓮縣':'23',
    '澎湖縣':'24',
    '金門縣':'25',
    '連江縣':'26',
}

In [4]:
client = requests.session()

In [7]:
def get_var(client,region,headers):
    index_url='https://rent.591.com.tw/'
    res = client.get(index_url,headers=headers)
    res.encoding = 'utf-8'
    res.cookies['urlJumpIp']= region_map[region]
    print(res.cookies.get_dict())
    soup = bs(res.text, "lxml")
    print(soup)

    totalRows = soup.select('#container > section.listBox > div > div.listLeft > div.page-limit > div > a.pageNext')[0]['data-total']
    CSRF_TOKEN = [i['content'] for i in soup.select('head > meta[\'name\']') if i['name']=="csrf-token"][0]
    return (totalRows ,CSRF_TOKEN)

In [8]:
get_var(client,'台北市',headers)

{'urlJumpIp': '1', 'new_rent_list_kind_test': '0', '591_new_session': 'eyJpdiI6IlF2blwvc2cxK2R0ZHp2MUpNXC8rbk9IZz09IiwidmFsdWUiOiIxQitHbFppaXJuR0pIU2hSSlphZldlZmp6M3I0M000Qzc3YnJUS3ZPenFURWkzUXpvaSsrcDVyRDRKSkpoZm8rTldva1BCUWJtZHNmVUFocU10enIzdz09IiwibWFjIjoiYjBkOGEyYmFlZDhjMjIzZWRmZTNlOTUyOGFlM2RkNGE2YjRjMDFmZjMyZmY2MDM5MmY2MzY1OGJmYWVlNDVhYiJ9'}
<!DOCTYPE HTML>
<html lang="zh-TW">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="zh-TW" http-equiv="content-language"/>
<meta content="數字科技股份有限公司" name="author"/>
<meta content="591.com.tw" name="copyright"/>
<meta content="IE=edge, chrome=1" http-equiv="X-UA-Compatible"/>
<title>【桃園市出租】-591房屋交易網</title>
<meta content="桃園市出租,出租,591,出租,591出租" name="keywords"/>
<meta content="591出租頻道,為您提供桃園市最豐富的資訊,還有整層住家,獨立套房,分租套房,雅房,其他出租資訊,讓您快速找到滿意的物件。找就上591！" name="description"/>
<meta content="index,follow" name="robots"/>
<meta content="index,follow,archive" name="googlebot"/>
<meta content="hEZ-1CusNSS7yOwwpax-

('7600', 'ApXOgpnhHLR4M2cLqBnREQEHoZeKhWn8V3eeej1F')

In [ ]:
def get_searchUrl(i,region,headers):
    params = {
        'is_new_list': 1,
        'type': 1,
        'kind':0,
        'searchtype':1,
        'region':region_map[region],
        'firstRow=':30*(i+1),
        'totalRows':get_var(client,region,headers)[0]
    }
    base_url = 'https://rent.591.com.tw/home/search/rsList?'
    url = base_url + urlencode(params)
    return url

In [ ]:
def get_searchUrlist(client,region,headers):
    vars = get_var(client,region,headers)
    headers['X-CSRF-Token'] = vars[1]
    for i in range(round(int(vars[0])/30)):
        searchUrl = get_searchUrl(i,region,headers)
        print(searchUrl)
        res_search = client.get(searchUrl,headers=headers)
        res_search.encoding = 'utf-8'
        data = json.loads(res_search.text)
        return data['data']['topData'][0]
#         rent_list = [i['id'] for i in data['data']['data']]
        

In [ ]:
get_searchUrlist(client,'台北市',headers)